# This notebook shows how to manage the events table with Spark

We first need to the kudu jars to the spark session

In [1]:
%%configure -f
{ "conf": {"spark.jars.packages": "org.apache.kudu:kudu-spark2_2.11:1.4.0" }}

In [2]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
13,application_1501855380192_0054,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@5f3783d5

In [3]:
import org.apache.kudu.spark.kudu._
import org.apache.spark.sql.functions._

//we need to define a udf function for converting two strings into a map
val asMap = udf((keys:String, values: String) => keys.split("#").zip(values.split("#")).toMap)

//then we define a first dataframe df1 associated to the kudu table
val df1 = spark.sqlContext.read.options(Map("kudu.master" -> "master:7051","kudu.table" -> "Events")).kudu

//the next dataframe contains an additional column of type Map containing a map for the two arrays: keys and values
val df2 = df1.withColumn("tags", asMap($"attributesKeys",$"attributesValues"))

df2: org.apache.spark.sql.DataFrame = [id: string, ts: bigint ... 12 more fields]

In [4]:
//let's show a couple of rows of df2
df2.select("id", "ts", "tags").take(10).toList

res9: List[org.apache.spark.sql.Row] = List([a4f68e7e-9dd6-4f26-864e-0d6b81a80d9a,1504151825780,Map(tag1 -> value1, tag2 -> value2)], [a4f68f1a-87b4-4794-91a3-a917fa50d7a9,1504136325911,Map(tag1 -> value1, tag2 -> value2)], [a4f68f69-747a-4da9-bfb4-b179dc661248,1504137698037,Map(tag1 -> value1, tag2 -> value2)], [a4f690a9-26af-4658-afb9-92f8c448e110,1504114696410,Map(tag1 -> value1, tag2 -> value2)], [a4f69320-44d6-4a18-af24-b136f323f821,1504113131620,Map(tag1 -> value1, tag2 -> value2)], [a4f6936e-f5eb-4b7f-a449-2c18bb96a0b2,1504187852601,Map(tag1 -> value1, tag2 -> value2)], [a4f6940f-fd31-4b97-9fe3-e8ca9b04d49b,1504197118273,Map(tag1 -> value1, tag2 -> value2)], [a4f69669-0833-4f65-9d7e-d260df7c4173,1504166699534,Map(tag1 -> value1, tag2 -> value2)], [a4f696b3-f11b-40de-a256-145bbf67...

In [7]:
df2.count()

res12: Long = 100000000